<a href="https://colab.research.google.com/github/JellyJoa/DataAnalysis/blob/master/Dacon/sound_classify/baseline2_insert2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import librosa 
import librosa.display as dsp
from IPython.display import Audio
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [2]:
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(929)

In [3]:
train = pd.read_csv('/content/drive/MyDrive/DL/sound_classify/train.csv')
train.head()

,file_name,label
0,001.wav,9
1,002.wav,0
2,004.wav,1
3,005.wav,8
4,006.wav,0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  400 non-null    object
 1   label      400 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 6.4+ KB


In [5]:
data, sample_rate = librosa.load('/content/drive/MyDrive/DL/sound_classify/train/001.wav', sr = 16000)
print('sample_rate:', sample_rate, ', audio shape:', data.shape)
print('length:', data.shape[0]/float(sample_rate), 'secs')

sample_rate: 16000 , audio shape: (10192,)
length: 0.637 secs


In [6]:
def train_dataset():
    folder = "/content/drive/MyDrive/DL/sound_classify/train/"
    dataset = []
    for file in tqdm(os.listdir(folder),colour='green'):
        if 'wav' in file:
            abs_file_path = os.path.join(folder,file)
            data, sr = librosa.load(abs_file_path, sr = 16000)
            class_label = int(train[train.file_name == file].label)
            dataset.append([data,class_label])
    
    print("Dataset 생성 완료")
    return pd.DataFrame(dataset,columns=['data','label'])

In [7]:
def test_dataset():
    folder = "/content/drive/MyDrive/DL/sound_classify/test/"
    dataset = []
    for file in tqdm(os.listdir(folder),colour='green'):
        if 'wav' in file:
            abs_file_path = os.path.join(folder,file)
            data, sr = librosa.load(abs_file_path, sr = 16000)
            
            dataset.append([data, file])
    
    print("Dataset 생성 완료")
    return pd.DataFrame(dataset,columns=['data', 'file_name'])

In [8]:
train_wav = train_dataset()
test_wav = test_dataset()

100%|██████████| 400/400 [00:12<00:00, 32.32it/s]


Dataset 생성 완료


100%|██████████| 200/200 [00:05<00:00, 33.86it/s]

Dataset 생성 완료


In [9]:
train_wav.head()

,data,label
0,"[3.6655838e-05, -3.7366447e-06, 3.4776433e-05,...",5
1,"[0.00011985076, 0.00016174652, 0.00017246709, ...",9
2,"[1.2653453e-05, 2.3892262e-05, -7.51332e-06, 4...",0
3,"[-0.00010586961, -0.00020532755, -0.0002140045...",7
4,"[-4.289015e-05, 9.891299e-05, 2.6636611e-05, 0...",4


In [10]:
train_x = np.array(train_wav.data)
test_x = np.array(test_wav.data)

In [11]:
def get_mini(data):

    mini = 9999999
    for i in data:
        if len(i) < mini:
            mini = len(i)

    return mini

train_mini = get_mini(train_x)
test_mini = get_mini(test_x)

#음성들의 길이를 맞춰줍니다.

mini = np.min([train_mini, test_mini])

In [12]:
print('가장 작은 길이 :', mini)

가장 작은 길이 : 5711


In [13]:
def set_length(data, d_mini):

    result = []
    for i in data:
        result.append(i[:d_mini])
    result = np.array(result)

    return result

train_x = set_length(train_x, mini)
test_x = set_length(test_x, mini)

In [14]:
print('train :', train_x.shape)
print('test :', test_x.shape)

train : (400, 5711)
test : (200, 5711)


In [15]:
extracted_features = librosa.feature.mfcc(y=train_x[0], sr=16000, n_mfcc=65)
extracted_features.shape

(65, 12)

In [16]:
def preprocess_dataset(data):
    mfccs = []
    for i in data:
        extracted_features = librosa.feature.mfcc(y=i,
                                              sr=16000,
                                              n_mfcc=65)
        mfccs.append(extracted_features)
            
    return mfccs

In [17]:
train_mfccs = preprocess_dataset(train_x)
train_mfccs = np.array(train_mfccs)
train_mfccs = train_mfccs.reshape(-1, train_mfccs.shape[1], train_mfccs.shape[2], 1)
#test_x = test_x.reshape(-1, test_x.shape[1], test_x.shape[2], 1)

In [18]:
np.array(train_mfccs).shape

(400, 65, 12, 1)

In [19]:
import torchvision.datasets as datasets # 데이터셋 집합체
import torchvision.transforms as transforms # 변환 툴

from torch.utils.data import DataLoader # 학습 및 배치로 모델에 넣어주기 위한 툴
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, X, y, train_mode=True, transforms=None): #필요한 변수들을 선언
        self.X = X
        self.y = y
        self.train_mode = train_mode
        self.transforms = transforms

    def __getitem__(self, index): #index번째 data를 return
        X = self.X[index]
        
        if self.transforms is not None:
            X = self.transforms(X)

        if self.train_mode:
            y = self.y[index]
            return X, y
        else:
            return X
    
    def __len__(self): #길이 return
        return len(self.X)

In [20]:
train_X = train_mfccs[:300]
vali_X = train_mfccs[300:]

In [21]:
train_y = train_wav.label[:300]
vali_y = train_wav.label[300:].reset_index(drop = True)

In [22]:
# 에포크 설정
num_epochs = 100

# 배치 사이즈 설정
batch_size = 10

#만든 train dataset를 DataLoader에 넣어 batch 만들기
train_dataset = CustomDataset(X=train_X, y=train_y)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)

vali_dataset = CustomDataset(X=vali_X, y=vali_y)
vali_loader = DataLoader(vali_dataset, batch_size = batch_size, shuffle=False)

In [23]:
train_batches = len(train_loader)
vali_batches = len(vali_loader)

print('/ total train batches :', train_batches)
print('/ total valid batches :', vali_batches)

/ total train batches : 30
/ total valid batches : 10


In [25]:
from tqdm.auto import tqdm
import torch.nn as nn # 신경망들이 포함됨

class CNNclassification(torch.nn.Module):
    def __init__(self):
        super(CNNclassification, self).__init__()
        self.layer1 = torch.nn.Sequential(
            nn.Conv2d(65, 10, kernel_size=2, stride=1, padding=1), #cnn layer
            nn.ReLU(), #activation function
            nn.MaxPool2d(kernel_size=2, stride=2)) #pooling layer
        
        self.layer2 = torch.nn.Sequential(
            nn.Conv2d(10, 100, kernel_size=2, stride=1, padding=1), #cnn layer
            nn.ReLU(), #activation function
            nn.MaxPool2d(kernel_size=2, stride=2)) #pooling layer
        
        self.layer3 = torch.nn.Sequential(
            nn.Conv2d(100, 200, kernel_size=2, stride=1, padding=1), #cnn layer
            nn.ReLU(), #activation function
            nn.MaxPool2d(kernel_size=2, stride=2)) #pooling layer
        
        self.layer4 = torch.nn.Sequential(
            nn.Conv2d(200, 300, kernel_size=2, stride=1, padding=1), #cnn layer
            nn.ReLU(), #activation function
            nn.MaxPool2d(kernel_size=2, stride=2)) #pooling layer
        
        self.fc_layer = nn.Sequential( 
            nn.Linear(300, 10) #fully connected layer(ouput layer)
        )    
        
    def forward(self, x):
        
        x = self.layer1(x) #1층
        
        x = self.layer2(x) #2층
         
        x = self.layer3(x) #3층
        
        x = self.layer4(x) #4층
        
        x = torch.flatten(x, start_dim=1) # N차원 배열 -> 1차원 배열
        
        out = self.fc_layer(x)
        return out

In [26]:
import torch.optim as optim # 최적화 알고리즘들이 포함힘

model = CNNclassification().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(params = model.parameters(), lr = 1e-3 )
scheduler = None

In [27]:
model(torch.rand(10, 65, 12, 1).to(device))

tensor([[-0.0002, -0.0490, -0.1010, -0.0371, -0.0610,  0.0279, -0.0544, -0.0124,
         -0.0470,  0.0099],
        [-0.0011, -0.0509, -0.1025, -0.0356, -0.0625,  0.0291, -0.0575, -0.0122,
         -0.0494,  0.0131],
        [-0.0009, -0.0487, -0.1009, -0.0352, -0.0585,  0.0279, -0.0533, -0.0102,
         -0.0491,  0.0119],
        [-0.0012, -0.0508, -0.1032, -0.0353, -0.0595,  0.0291, -0.0578, -0.0103,
         -0.0493,  0.0134],
        [-0.0031, -0.0496, -0.1034, -0.0359, -0.0597,  0.0300, -0.0569, -0.0137,
         -0.0477,  0.0131],
        [-0.0015, -0.0520, -0.1030, -0.0371, -0.0625,  0.0279, -0.0542, -0.0137,
         -0.0484,  0.0120],
        [-0.0016, -0.0490, -0.1048, -0.0342, -0.0597,  0.0281, -0.0591, -0.0122,
         -0.0461,  0.0114],
        [-0.0029, -0.0514, -0.1036, -0.0371, -0.0596,  0.0271, -0.0554, -0.0117,
         -0.0488,  0.0128],
        [-0.0024, -0.0501, -0.1054, -0.0368, -0.0619,  0.0308, -0.0569, -0.0120,
         -0.0473,  0.0130],
        [-0.0022, -

In [28]:
from tqdm.auto import tqdm

def train(model, optimizer, train_loader, scheduler, device): 
    model.to(device)
    n = len(train_loader)
    best_acc = 0
    
    for epoch in range(1,num_epochs): #에포크 설정
        model.train() #모델 학습
        running_loss = 0.0
        
        for wav, label in tqdm(iter(train_loader)):
            
            wav, label = wav.to(device), label.to(device) #배치 데이터
            optimizer.zero_grad() #배치마다 optimizer 초기화
        
            # Data -> Model -> Output
            logit = model(wav) #예측값 산출
            loss = criterion(logit, label) #손실함수 계산
            
            # 역전파
            loss.backward() #손실함수 기준 역전파 
            optimizer.step() #가중치 최적화
            running_loss += loss.item()
             
        print('[%d] Train loss: %.10f' %(epoch, running_loss / len(train_loader)))
        
        if scheduler is not None:
            scheduler.step()
            
            
        #Validation set 평가
        model.eval() #evaluation 과정에서 사용하지 않아야 하는 layer들을 알아서 off 시키도록 하는 함수
        vali_loss = 0.0
        correct = 0
       
        with torch.no_grad(): #파라미터 업데이트 안하기 때문에 no_grad 사용
            for wav, label in tqdm(iter(vali_loader)):
                
                wav, label = wav.to(device), label.to(device)
                logit = model(wav)
                vali_loss += criterion(logit, label)
                pred = logit.argmax(dim=1, keepdim=True)  #10개의 class중 가장 값이 높은 것을 예측 label로 추출
                correct += pred.eq(label.view_as(pred)).sum().item() #예측값과 실제값이 맞으면 1 아니면 0으로 합산
        vali_acc = 100 * correct / len(vali_loader.dataset)
        print('Vail set: Loss: {:.4f}, Accuracy: {}/{} ( {:.0f}%)\n'.format(vali_loss / len(vali_loader), correct, len(vali_loader.dataset), 100 * correct / len(vali_loader.dataset)))
        
        #베스트 모델 저장
        if best_acc < vali_acc:
            best_acc = vali_acc
            torch.save(model.state_dict(), '/content/drive/MyDrive/DL/sound_classify/saved/best_model2.pth') #이 디렉토리에 best_model.pth을 저장
            print('Model Saved.')

In [29]:
train(model, optimizer, train_loader, scheduler, device)

  0%|          | 0/30 [00:00<?, ?it/s]

[1] Train loss: 2.3211733103


  0%|          | 0/10 [00:00<?, ?it/s]

Vail set: Loss: 2.2671, Accuracy: 7/100 ( 7%)

Model Saved.


  0%|          | 0/30 [00:00<?, ?it/s]

...

Vail set: Loss: 0.2755, Accuracy: 90/100 ( 90%)



  0%|          | 0/30 [00:00<?, ?it/s]

[99] Train loss: 0.1195450066


  0%|          | 0/10 [00:00<?, ?it/s]

Vail set: Loss: 0.2697, Accuracy: 86/100 ( 86%)



In [30]:
test_mfccs = preprocess_dataset(test_x)
test_mfccs = np.array(test_mfccs)
test_mfccs = test_mfccs.reshape(-1, test_mfccs.shape[1], test_mfccs.shape[2], 1)

In [31]:
test_mfccs.shape

(200, 65, 12, 1)

In [32]:
def predict(model, test_loader, device):
    model.eval()
    model_pred = []
    with torch.no_grad():
        for wav in tqdm(iter(test_loader)):
            wav = wav.to(device)

            pred_logit = model(wav)
            pred_logit = pred_logit.argmax(dim=1, keepdim=True).squeeze(1)

            model_pred.extend(pred_logit.tolist())
    return model_pred

In [33]:
test_dataset = CustomDataset(X=test_mfccs, y= None, train_mode=False)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False)

In [34]:
# Validation Accuracy가 가장 뛰어난 모델을 불러옵니다.
checkpoint = torch.load('/content/drive/MyDrive/DL/sound_classify/saved/best_model2.pth')
model = CNNclassification().to(device)
model.load_state_dict(checkpoint)

# Inference
preds = predict(model, test_loader, device)
preds[0:5]

  0%|          | 0/20 [00:00<?, ?it/s]

[2, 7, 8, 1, 8]

In [35]:
len(preds)

200

In [36]:
test_wav['label'] = preds
test_wav = test_wav[['file_name', 'label']]

pred_df = test_wav.copy()
pred_df = pred_df.sort_values(by=[pred_df.columns[0]], ascending=[True]).reset_index(drop=True)
pred_df.head()

,file_name,label
0,003.wav,0
1,008.wav,9
2,010.wav,8
3,015.wav,8
4,024.wav,2


In [37]:
submission = pd.read_csv('/content/drive/MyDrive/DL/sound_classify/submission.csv')
submission['label'] = pred_df['label']
submission.head()

,file_name,label
0,003.wav,0
1,008.wav,9
2,010.wav,8
3,015.wav,8
4,024.wav,2


In [38]:
submission.to_csv('/content/drive/MyDrive/DL/sound_classify/submit2-6.csv', index=False)